<a href="https://colab.research.google.com/github/zjuiEMLab/rshub/blob/main/demo/Soil-demo-AIEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Active Soil demo

In [ ]:
import datetime
import copy
!pip install rshub -q

In [ ]:
# Define user token
token = 'ENTER YOUR TOKEN HERE' # Register an account to get a token
# Chang your task name or project name every time you run a new job
project_name = 'Demo'
task_name1 = 'AIEM'

### Step 1: Define Scenario flag

In [ ]:
# ============== CHANGE YOUR INPUT PARAMETERS HERE ==============
# ====== Parameters not define will be set to default values ======

# 'soil: Bare soil
# 'snow: Snow
# 'veg: Vegetation covered soil
scenario_flag = 'soil'

### Step 2: Define observation description

In [ ]:
# 1) Observation mode
# 'bs': Active (Backscatter)
# 'tb': Passive (Brightness temperature)
output_var = 'sigma' # for soil model, both active and passive results will be outputed; Use this flag to retrieve results

# 2) Observation characteristics
fGHz = 1.26

### Step 3: Define Algorithm flag

In [ ]:
# 1: VIE NMM3D
algorithm = 'aiem'

### Step 4: Describe your scenario

In [ ]:
#
theta_i_deg = [30,40] #incident angle in degree.
theta_s_deg = theta_i_deg # scattering angle in degree
phi_s_deg = 12.2034 # scattering azimuth angle in deg while  incident azimuth angle is 0 degree
phi_i_deg = 0 # incident azimuth angle
kl = 0.2955 # normalized surface correlation length multiplies by wave number k.
ks = 0.2955 # normalized surface rms height multiplies by wave number k
perm_soil_r = 10.0257 # the real part of surface relative dielectric constant
perm_soil_i = 1.1068 # the imaginary part of surface relative dielectric constant
rough_type = 2 # 1 Gaussian; 2 exponential; 3 transformed exponential correlation (1.5-power)


In [ ]:
data = {
    'scenario_flag': scenario_flag,
    'output_var': output_var,'fGHz': fGHz,
    'algorithm':algorithm,
    #'h': h,'Ts':Ts,'Tg':Tg,
    #'epsr_ice_re':epsr_ice_re,'epsr_ice_im':epsr_ice_im,
    'theta_i_deg':theta_i_deg,'theta_s_deg':theta_s_deg,'phi_s_deg':phi_s_deg,'phi_i_deg':phi_i_deg,'kl':kl,
    'ks':ks,'perm_soil_r':perm_soil_r,'perm_soil_i':perm_soil_i,'rough_type':rough_type,
    'project_name':project_name,
    'task_name':task_name1,
    'token': token,
    'force_update_flag':1 # force replace existing task 
}

## Run models

In [ ]:
from rshub import submit_jobs
result=submit_jobs.run(data)

In [ ]:
print(result['result'])

In [ ]:
# Store log information
now = datetime.datetime.now()
logname = 'log_' + now.strftime("%Y%m%d%H%M%D") + '.txt'
f = open('log.txt',"a")
head_string = '======' + now.strftime("%Y%m%d%H%M%D") + '======' + "\n"
f.write(head_string)
f.write(f' project_name = {project_name} \n')
f.write(f' task_name = {task_name1} \n')
f.write(f' data = {data} \n')
f.write('================================ \n')
f.close()

# Check Job Status (It will take over half an hour!)

In [ ]:
from rshub import submit_jobs
result=submit_jobs.check_completion(token, project_name, task_name1)
print(result)

In [ ]:
from rshub import load_file

data = load_file(token, project_name, task_name1,scenario_flag=scenario_flag,algorithm=algorithm,output_var=output_var)
message = data.load_error_message()


# Post Process

In [ ]:
from rshub import load_file

## Active

In [ ]:
# load mat file with project id, frequencies,variables to load
output_var = 'sigma'
data = load_file(token, project_name, task_name1,scenario_flag=scenario_flag,algorithm=algorithm,output_var=output_var)

data_active = data.load_outputs(fGHz=fGHz, inc_ang=40) #

In [ ]:
# Backscatter in order of VV, HV, VH, HH
HH = data_active['HH']
VV = data_active['VV']
HV = data_active['HV']
VH = data_active['VH']

print(f"VV:{VV[0][0][0][0]} dB, HV:{HV[0][0][0][0]} dB, VH:{VH[0][0][0][0]} dB, HH:{HH[0][0][0][0]} dB")